Project by Jeremy Bouhi & Lucas Trevalinet

# Classification Document


## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import re

## STEP 1 : Grab Data

In [2]:
# We consider you had already unzip the dataset file and that this file is in the parent folder

#GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global") #Lucas' Path
GLOBAL_PATH = os.path.join("..","20180315-170000", "juri","cass","global") #Jeremy's Path

# We need to do that for excluding data from Juri_path
CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

In [4]:
ident = []
text = []
division = []
DATA_PATH = [CIVILE_PATH] # For faster test
#DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

for DIVISION_PATH in DATA_PATH :
    xml_files = list(Path(DIVISION_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)
                
            # For getting the text
            #for child in root.iter('BLOC_TEXTUEL'):
             #   contenu = re.sub('<[^<]+?>', '', ET.tostring(child).decode("utf-8"))
              #  text.append(contenu)
            
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = "".join(child.itertext())
                text.append(contenu)
                
            # For getting the division    
            for child in root.iter('META_JURI_JUDI'):
                formation = re.sub('CHAMBRE|_|[0-9]', '', child.find('FORMATION').text)
                division.append(formation)
                
            

In [5]:
print ("text: ", len(text))
print ("ID: ", len(ident))
print ("division: ", len(division))

text:  59250
ID:  59250
division:  59250


## STEP 2 : Create DataFrame to manipulate easily data

In [6]:
d = {'id': ident, 'text': text, 'division': division}
df = pd.DataFrame(data = d)

#print('df', df)

df["text"]

0        \nLA COUR DE CASSATION, TROISIÈME CHAMBRE CIVI...
1        \nLA COUR DE CASSATION, TROISIÈME CHAMBRE CIVI...
2        \nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVIL...
3        \nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVIL...
4        \nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVIL...
5        \nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVIL...
6        \n\nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIV...
7        \nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVIL...
8        \nLA COUR DE CASSATION, PREMIÈRE CHAMBRE CIVIL...
9        \nLA COUR DE CASSATION, PREMIÈRE CHAMBRE CIVIL...
10       \nLA COUR DE CASSATION, PREMIÈRE CHAMBRE CIVIL...
11       \nLA COUR DE CASSATION, TROISIÈME CHAMBRE CIVI...
12       \nLA COUR DE CASSATION, TROISIÈME CHAMBRE CIVI...
13       \nLA COUR DE CASSATION, TROISIÈME CHAMBRE CIVI...
14       \nLA COUR DE CASSATION, TROISIÈME CHAMBRE CIVI...
15       \nLA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVIL...
16       \nLA COUR DE CASSATION, PREMIÈRE CHAMBRE CIVIL.

## STEP 3 : Split Data

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test= train_test_split(df["text"],df["division"], test_size = 0.2 ,random_state=42,stratify=df["division"] )
print(X_train.head())

19298    \nSur le moyen unique du pourvoi incident, qui...
31218    \nSur le moyen unique :\n   Vu l'article 2127 ...
28634    \n\n   Sur le moyen unique : \n\n   Attendu qu...
35146                                                 \n\n
22017    \nSur la recevabilité du pourvoi contestée par...
Name: text, dtype: object


In [61]:
print("Training size : ",X_train.shape[0])
print("Testing size : ",X_test.shape[0])

Training size :  47400
Testing size :  11850


## STEP 4 : Clean Data

In [62]:
def clean_references_to_law_codes(text) : 
    return re.sub('(?<=Code )civil|de l\'action sociale et des familles|de l\'artisanat|des assurances|de l\'aviation civile|du cinéma et de l image animée|de commerce|des communes( de la Nouvelle-Calédonie)?|de la consommation', '', text)

X_train = X_train.apply(clean_references_to_law_codes)

In [65]:
def clean_references_to_law_articles(text) :
    return re.sub('(?:[A-Z][\.-]? ?)?(?:\d+-?\d+)', 'N', text)

X_train = X_train.apply(clean_references_to_law_articles)

In [66]:
# Preprocessing step pull off all the htlm tag, "\n", and any special characters
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[.,\/#!$%\^&\*;:{}=\-_`~()«»°–]','', text)
    text = text.replace('\n','').replace('\'',' ')
    return text.replace('\s{2,}',' ')

X_train = X_train.apply(preprocessor)

In [67]:
# Set to lower case
def to_lower_case(s) :
    return s.lower()

X_train = X_train.apply(to_lower_case)

In [68]:
# We decided to replace any corresponding labels to a unique special label : *_* 
def clean_words_corresponding_to_labels(text) :
    return re.sub('criminel(le)?|commercial(e)?|social(e)?|civil(e)?', '*_*', text)

X_train = X_train.apply(clean_words_corresponding_to_labels)

In [45]:
X_train[4]

'la cour de cassation deuxième chambre *_* a rendu l arrêt suivant  sur le premier moyen pris en sa seconde branche  vu l article n du titre ii de la loi des n août n et le décret du n fructidor an ii et l article n du code de la sécurité *_* ensemble les articles n et n du code de l action *_* et des familles dans leur rédaction applicable à la date des soins litigieux  attendu que les litiges nés de la facturation aux caisses d assurance maladie par les établissements de santé de la part des dépenses prises en charge par l etat pour les soins dispensés au titre des soins urgents dans les conditions prévues par les deux derniers de ces textes se rapportent à l attribution des prestations d aide *_* de l etat et ne sont pas dès lors au nombre des différends auxquels donne lieu au sens du troisième l application des législations et réglementations de sécurité *_* et de mutualité *_* agricole  attendu selon le jugement attaqué rendu en dernier ressort qu ayant dispensé au titre des soins

In [69]:
#Stop_words :
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def clean_data(text):
    
    french_stopwords = set(stopwords.words('french'))
    tokens = word_tokenize(text, language='french')
    
    content_tokens = ""
    for token in tokens:
        if token not in french_stopwords:
            content_tokens += token
            content_tokens += " "
    return(content_tokens) 

X_train = X_train.apply(clean_data)

## Most influential words

In [73]:
import collections

wordcount = {}

for word in X_train[4].split():
    if word not in wordcount:
        wordcount[word] = 1
    else:
        wordcount[word] += 1
            
word_counter = collections.Counter(wordcount)
for word, count in word_counter.most_common(10):
    print(word, ": ", count)

les :  35
*_* :  22
soins :  19
aide :  13
article :  12
code :  12
jugement :  12
a :  11
urgents :  11
strasbourg :  9


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer()
np.set_printoptions(precision=2)

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tfidf.fit_transform(vectorizer.fit_transform(X_train)).toarray()

MemoryError: 

## STEP 5 : Train the model

In [21]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [22]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

ValueError: could not convert string to float: "\nSur le moyen unique du pourvoi incident, qui est préalable :\n   Attendu, selon l'arrêt attaqué qu'un tribunal de grande instance a prononcé la séparation de corps des époux X... aux torts de l'épouse ; que, par assignation du 21 juillet 1989, le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce ; que, par assignation en date du 2 août 1989, Mme X..., autorisée par une ordonnance de non-conciliation du 19 janvier 1989, a assigné son mari en divorce pour faute ; qu'après jonction de ces deux procédures, le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme, et, accueillant la demande en divorce de celle-ci, prononcé également le divorce à son profit ; qu'ensuite le Tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés ;\n   Attendu qu'il est fait grief à l'arrêt infirmatif attaqué d'avoir déclaré irrecevable la demande en 

In [153]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)

ada_clf= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

gdb_clf = GradientBoostingClassifier(random_state=42, n_estimators=100)

voting_clf = VotingClassifier(
    estimators=[('rfc', rnd_clf), ('abc', ada_clf), ('gbr', gdb_clf)],
    voting='hard')

In [154]:
for clf in (rnd_clf, ada_clf, gdb_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

ValueError: could not convert string to float: 'sur le moyen unique du pourvoi incident qui est préalable    attendu selon l arrêt attaqué qu un tribunal de grande instance a prononcé la séparation de corps des époux x aux torts de l épouse  que par assignation du 21 juillet 1989 le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce  que par assignation en date du 2 août 1989 mme x autorisée par une ordonnance de nonconciliation du 19 janvier 1989 a assigné son mari en divorce pour faute  qu après jonction de ces deux procédures le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme et accueillant la demande en divorce de celleci prononcé également le divorce à son profit  qu ensuite le tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés    attendu qu il est fait grief à l arrêt infirmatif attaqué d avoir déclaré irrecevable la demande en divorce de la femme alors que d 